In [6]:
import numpy as np
import pandas as pd
from scipy import stats
import time
import evcouplings
#from evcouplings.align import Alignment # install this using 'pip install evcouplings'
import glob


In [7]:
# Read in list of isolates
phenotypes =  pd.read_csv("../input_data/master_table_resistance.csv")
isolate_order = list(phenotypes.Isolate)

/tmp/ipykernel_4523/1013361237.py:2: DtypeWarning: Columns (3,4,7,8,9,12,17,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  phenotypes =  pd.read_csv("../input_data/master_table_resistance.csv")


In [11]:
# Iterate through each fasta file, select only the positions with a high enough MAF
# Keep only the isolates with phenotype data, reorder the alignment to match isolate_order
# save a reduced fasta file
!mkdir fastas_reduced
path_to_fastas = "../input_data/fasta_files"
for f in glob.glob(f"{path_to_fastas}/*.fasta"):
    print(f)
    # Reads the alignment file
    aln = Alignment.from_file(open(f))
    
    # Impose a minor allele frequency threshold of 0.001 in order to keep computation tractable
    # NOTE: the order in which you do the MAF thresholding and then subsetting the isolates matters!
    # Here it's fine because the only isolates in my fasta files are the ones that have phenotypes
    indices_to_keep = np.where(aln.frequencies.max(axis=1) < .999)[0]
    subset_alignment = aln.select(indices_to_keep)
    
    print("original alignment shape", aln.matrix.shape)
    print("reduced alignment shape", subset_alignment.matrix.shape)
    # Cleanup giant variables
    del aln

    # Get a list of the ids we are keeping, in order
    
    # First, correct the ids in the alignment so that they match the table of phenotypes
    subset_alignment.ids = [x.split("/")[-1].split(".")[0] for x in subset_alignment.ids]
    subset_alignment.id_to_index = {x:idx for idx,x in enumerate(subset_alignment.ids)}
    
    # Get the indices that would correctly reorder the alignment to match isolate_order
    reorder_index = [
        subset_alignment.id_to_index[x] for x in isolate_order if x in subset_alignment.id_to_index
    ]
    
    # Reorder based on reorder_index
    subset_alignment.ids = list(np.array(subset_alignment.ids)[reorder_index])
    subset_alignment.matrix = subset_alignment.matrix[reorder_index, :]
    print("shape with isolates ordered", subset_alignment.matrix.shape)
    
    # Get the name of the fasta file for saving
    name = f.split("/")[-1].split("_")[0]
    
    # save the reduced file
    subset_alignment.write(open(f"fastas_reduced/{name}_reduced.fa", "w"))
    np.save(f"fastas_reduced/{name}_indices.npy", indices_to_keep)
    
# Save the isolate indices in order, just in case
pd.DataFrame(subset_alignment.ids).to_csv("isolate_indexes.csv")

mkdir: cannot create directory ‘fastas_reduced’: File exists


NameError: name 'subset_alignment' is not defined

In [23]:
path_to_fastas = "../input_data/fasta_files"
for f in glob.glob(f"{path_to_fastas}/*.fasta"):
    print(f)
    aln = Alignment.from_file(open(f))